In [1]:
from __future__ import print_function
from random import randint
from itertools import product, starmap
# product is essentially combinations without repetitions
# starmap is an iterator that applies a function over a set of tuples (generated by product)

import numpy as np
import os

In [2]:
GRID = np.zeros((10,10), dtype=str)
GRID

array([['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', '']], dtype='|S1')

In [3]:
GRID.shape

(10, 10)

In [2]:
def mhd(tileA, tileB):
    """Computes the Manhattan distance between two (x,y) tiles"""
    return abs(tileA[0] - tileB[0]) + abs(tileA[1] - tileB[1])

def place_goal(shape, min_dist=5):
    """Places the Goal and Subject tile in a grid so that they are at a minimum distance"""
    grid = np.zeros(shape, dtype=str)
    max_x, max_y = [p - 1 for p in shape]
    subj_pos, goal_pos = (0,0), (0,0)
    
    while mhd(subj_pos, goal_pos) < min_dist:
        subj_pos = (randint(0, max_x), randint(0, max_y))
        goal_pos = (randint(0, max_x), randint(0, max_y))
        
    grid[subj_pos] = "S"
    grid[goal_pos] = "G"
    
    return (grid, subj_pos, goal_pos)

In [64]:
g1, subj_pos, goal_pos = place_goal((5,5), 6)
g1

array([['', '', '', '', ''],
       ['G', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', 'S']], dtype='|S1')

In [4]:
def neighbors(shape, pos, rad=1):
    """Gets the set of surrounding tiles given a tile and its set radius"""
    max_X, max_Y = shape
    
    jumps = [0] + [-x for x in range(1, rad+1)] + [x for x in range(1, rad+1)]
    area = list(starmap(lambda a,b: (pos[0] + a, pos[1] + b), product(jumps, jumps)))
    
    return [p for p in area[1:] if (p[0] in range(max_X) and p[1] in range(max_Y))]

def place_drone(grid, subj_pos, dist=1):
    """Places the drone at a certain distance from the subject"""
    block = neighbors(grid.shape, subj_pos, dist)
    dr_pos = block[randint(0, len(block) - 1)]
    grid[dr_pos] = "D"
    return (grid, dr_pos)

In [65]:
g1, drone_pos = place_drone(g1, subj_pos, 1)
g1

array([['', '', '', '', ''],
       ['G', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', 'D'],
       ['', '', '', '', 'S']], dtype='|S1')

In [9]:
X = np.argwhere(g1 == '')
X1 = [(x,y) for (x,y) in X if x == 3]
X1

[(3, 0), (3, 1), (3, 2), (3, 3), (3, 4)]

In [23]:
g1

array([['', '', '', '', 'G'],
       ['', '', '', '', ''],
       ['S', 'D', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', '']], dtype='|S1')

In [154]:
X1

[(3, 0), (3, 1), (3, 2), (3, 3), (3, 4)]

In [157]:
idX, idY = grp_range(X1, common_y=(False,3))
g1[idX, idY]

array(['', '', '', '', ''], dtype='|S1')

In [164]:
groups = []
maxl = max(*g1.shape)

for i in range(maxl):
    lX = group_by_adj([(x,y) for (x,y) in X if x == i], common_y=(False, i))
    groups += (lX)
    

for i in range(maxl):
    lY = group_by_adj([(x,y) for (x,y) in X if y == i], common_y=(True, i))
    groups += (lY)

print([(len(g), g) for g in groups])

[(3, ([(0, 0), (0, 1), (0, 2), (0, 3)], 0, [0, 1, 2, 3])), (3, ([(1, 0), (1, 1), (1, 2), (1, 3), (1, 4)], 1, [0, 1, 2, 3, 4])), (3, ([(2, 2), (2, 3), (2, 4)], 2, [2, 3, 4])), (3, ([(3, 0), (3, 1), (3, 2), (3, 3), (3, 4)], 3, [0, 1, 2, 3, 4])), (3, ([(4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], 4, [0, 1, 2, 3, 4])), (3, ([(0, 0), (1, 0)], [0, 1], 0)), (3, ([(3, 0), (4, 0)], [3, 4], 0)), (3, ([(0, 1), (1, 1)], [0, 1], 1)), (3, ([(3, 1), (4, 1)], [3, 4], 1)), (3, ([(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)], [0, 1, 2, 3, 4], 2)), (3, ([(0, 3), (1, 3), (2, 3), (3, 3), (4, 3)], [0, 1, 2, 3, 4], 3)), (3, ([(1, 4), (2, 4), (3, 4), (4, 4)], [1, 2, 3, 4], 4))]


In [125]:
X3 = choose_path(X, g1.shape, 5)
X3

[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4)]

In [170]:
X4 = groups[0]
print(X4)
print(g1[X4[1], X4[2]])

([(0, 0), (0, 1), (0, 2), (0, 3)], 0, [0, 1, 2, 3])
['' '' '' '']


In [63]:
def dist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def grid_range(init_pos, end_pos):
    """
    Computes the range of the pedestrian path or wall width.
    :param init_pos:
    :param end_pos:
    :returns: A tuple (line, (init_x, init_y), (end_x, end_y)) where 'line'
              is a tuple (X, -1) or (-1, X) being 'X' the common coordinate
    """
    if init_pos[0] == end_pos[0]:
        # line'd be a row
        line, init, end = init_pos[0], init_pos[1], end_pos[1]
    elif init_pos[1] == end_pos[1]:
        # line'd be a col
        line, init, end = init_pos[1], init_pos[0], end_pos[0]
    return line, init, end

def grp_range(group, common_y):
    """"""
    is_y, idx = common_y
    if is_y:
        idx_x = [x for (x,_) in group]
        idx_y = idx
    else:
        idx_x = idx
        idx_y = [y for (_,y) in group]
        
    return (idx_x, idx_y)

def group_by_adj(ls, min_length=2, max_length=1000, common_y=False):
    """Groups elements whose two-by-two distance is 1, i.e. are adjacent."""
    
    if type(ls) is not list or len(ls) == 0:
        return []
    
    groups = []
    curr_group = [ls[0]]
    xs = ls[1:]
    
    while len(xs) > 0:
        d = dist(curr_group[-1], xs[0])
        # print("dist({}, {}) is {}".format(curr_group[-1], xs[0], d))
        
        if d == 1:
            # If the next node is adjacent, add it to the group
            curr_group.append(xs[0])
            # print("Current expanded to {}".format(curr_group))
        else:
            # Otherwise, close the current group and create a new one
            idx_x, idx_y = grp_range(curr_group, common_y)
            groups.append((curr_group, idx_x, idx_y))
            # print("New group added: {}".format(groups))
            curr_group = [xs[0]]
        xs = xs[1:]
        
    # return [g for g in groups if len(g) >= min_length and len(g) <= max_length]
    idx_x, idx_y = grp_range(curr_group, common_y)
    groups.append((curr_group, idx_x, idx_y))
    return [(g,x,y) for (g,x,y) in groups if len(g) in range(min_length, max_length)]

def choose_path(empties, shape, min_length=2):
    """
    Returns a path tuple whose length is at maximum, max_length.
    :returns: A tuple (line, init, end)
    """
    paths = []
    max_length = max(*shape)
    
    # Rows
    for rn in range(shape[0]):
        curr_row_groups = group_by_adj([(x,y) for (x,y) in empties if x == rn], min_length, max_length, common_y=(False, rn))
        paths += curr_row_groups
        
    # Columns
    for cn in range(shape[1]):
        curr_col_groups = group_by_adj([(x,y) for (x,y) in empties if y == cn], min_length, max_length, common_y=(True, cn))
        paths += curr_col_groups
        
    rnd = paths[randint(0, len(paths) - 1)]
    print(rnd)
    return rnd
    
def place_peds(grid, no_peds, subj_pos, drone_pos, goal_pos):
    """
    Place a limited number of pedestrians in the grid. Subject, Drone and Goal poses cannot
    be overwritten by the pedestrians
    """
    empties = np.argwhere(grid == '')
    
    peds = dict()
    for p in range(no_peds):
        nm = "P{}".format(p)
        peds[nm] = {}
        peds[nm]["pos"] = (line, pX, pY) = choose_path(empties, grid.shape) # (init:line:end, row(s), col(s))
        peds[nm]["loop"] = bool(randint(0,1))
        
        # Writes "·" in the points between
        print(pX, pY)
        
        grid[pX, pY] = "o"
        # Writes "PX" in the init and start of the ped path
        grid[line[0]] = grid[line[-1]] = nm
        
    return (grid, peds)

In [66]:
g1, peds_paths = place_peds(g1, 1, subj_pos, drone_pos, goal_pos)
g1

([(4, 0), (4, 1), (4, 2), (4, 3)], 4, [0, 1, 2, 3])
4 [0, 1, 2, 3]


array([['', '', '', '', ''],
       ['G', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', 'D'],
       ['P', 'o', 'o', 'P', 'S']], dtype='|S1')

In [44]:
for i in range(int(5 * 0.4)):
    print(i)

0
1


In [49]:
X = np.argwhere(g1 == "")
print(X)
print(X[randint(0, len(X) - 1)])

[[4 0]
 [4 1]
 [4 2]
 [4 3]
 [4 4]]
[4 0]


In [53]:
g1[4, 0]

''

In [67]:
def place_objs(grid, probs, max_objs):
    """
    Place a limited number of objects, depending on the probabilities of each object type.
    :param grid:
    :param probs: A dictionary holding probabilities for Trees,Walls,Doors
    :param max_objs:
    """
    
    trees, doors, walls = dict(), dict(), dict()
    tree_rng = int(max_objs * probs["tree"])
    door_rng = int(max_objs * probs["door"])
    wall_rng = int(max_objs * probs["wall"])
    
    for tn in range(tree_rng):
        empties = np.argwhere(grid == '')
        nm = "T{}".format(tn)
        # places the tree in an empty cell
        trees[nm] = dict(
            pos = empties[randint(0, len(empties) - 1)]
        )
        pX, pY = trees[nm]["pos"]
        grid[pX, pY] = "T"
        
    for dn in range(door_rng):
        empties = np.argwhere(grid == '')
        nm = "D{}".format(dn)
        # places the door in an empty cell with a random direction (only two: North/South and West/East)
        doors[nm] = dict(
            pos = empties[randint(0, len(empties) - 1)],
            ori = ["N", "W"][randint(0,1)]
        )
        pX, pY = doors[nm]["pos"]
        grid[pX, pY] = "D"
        
    for wn in range(wall_rng):
        empties = np.argwhere(grid == '')
        nm = "W{}".format(tn)
        walls[nm] = dict(
            pos = choose_path(empties, grid.shape) # returns (init:line:end)
        )
        # Writes "W" in the whole wall path
        _, pX, pY = walls[nm]["pos"]
        grid[pX, pY] = "W"

    return (grid, trees, doors, walls)

In [10]:
g1

array([['', '', 'P', 'D', 'S'],
       ['', '', 'o', '', ''],
       ['', '', 'o', '', ''],
       ['G', '', 'o', '', ''],
       ['', '', 'P', '', '']], dtype='|S1')

In [16]:
empties = np.argwhere(g1 == "")
X0 = [(x,y) for (x,y) in empties if x == 0]
X0

[(0, 0), (0, 2), (0, 3), (0, 4)]

In [18]:
group_by_adj(X0, common_y=(False, 0))

[([(0, 2), (0, 3), (0, 4)], 0, [2, 3, 4])]

In [68]:
obj_probs = {"tree": 0.4, "door": 0.0, "wall": 0.2}
g1, trees, doors, walls = place_objs(g1, obj_probs, 5)

([(0, 4), (1, 4), (2, 4)], [0, 1, 2], 4)


In [69]:
g1

array([['', '', '', '', 'W'],
       ['G', '', '', '', 'W'],
       ['', '', '', '', 'W'],
       ['T', '', '', 'T', 'D'],
       ['P', 'o', 'o', 'P', 'S']], dtype='|S1')

In [15]:
print(drone_pos, subj_pos, goal_pos)
print()
print(trees)
print(doors)
print(walls)
print(peds_paths)

(0, 3) (0, 4) (3, 0)

{'T0': {'pos': array([0, 1])}, 'T1': {'pos': array([2, 1])}}
{}
{'W1': {'pos': ([(1, 3), (2, 3), (3, 3), (4, 3)], [1, 2, 3, 4], 3)}}
{'P0': {'pos': ([(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)], [0, 1, 2, 3, 4], 2), 'loop': True}}


In [35]:
import pickle as pk

with open("example_grid2.pk", "wb") as f:
    pk.dump(g1, f)

---

In [23]:
import pickle as pk

with open("example_grid2.pk", "rb") as f:
    g1 = pk.load(f)

In [24]:
g1

array([['', '', 'T', '', 'G'],
       ['P', 'o', 'o', 'o', 'P'],
       ['S', 'T', '', '', ''],
       ['D', 'W', 'W', 'W', 'W'],
       ['', '', '', '', '']], dtype='|S1')

---
## A* pathfinder

  + The only thing the path finder cannot do is to jump walls or cross doors in the wrong direction.
  + It cannot cross the Drone cell `[D]` neither.
  + It can collide with pedestrian paths and avoid trees by rounding them.
  + It must start at the Subject cell `[S]` and end in the Goal cell `[G]`.
  + **We can only move horizontally or vertically one cell at a time**.
  
A future goal would be to modify the speed and stop duration for one or more points in the path. **This could also be placed randomly!**

In [80]:
class PathNode:
    def __init__(self, parent=None, position=None, cell_type="", weight=0):
        self.parent = parent
        self.pos = position
        self.weight = weight
        self.cell_type = cell_type
        
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.pos == other.pos
    
    def __repr__(self):
        return "Node({}, {}, {})".format(self.parent, self.pos, self.f)
    
    def __str__(self):
        return self.__repr__()

In [37]:
x = PathNode(None, (1,0))
x

Node(None, (1, 0), 0)

In [82]:
class PathFinder:
    def __init__(self, grid, init, end):
        """
        :param grid:
        :param init:
        :param end:
        """
        self.grid = grid
        self.init = init
        self.end = end
        self.path = None
        
        # self.a_star()
        
    @staticmethod
    def walkable_node_neighbors(grid, curr_node, rad=1):
        """Gets the set of surrounding tiles given a tile and its set radius"""
        
        max_X, max_Y = grid.shape
        my_X, my_Y = curr_node.pos
        # Drops the 4 diagonal surrounding tiles
        area = list(map(lambda (a,b): (a + my_X, b + my_Y), [(-1,0), (1,0), (0,-1), (0,1)]))

        within_range = (p for p in area if (p[0] in range(max_X) and p[1] in range(max_Y)))
        cell_types = (PathFinder.check_type(grid, cell) for cell in within_range)
        return [PathNode(curr_node, pos, ctype, weight) for (pos, walkable, weight, ctype) in cell_types if walkable]
        
    @staticmethod
    def check_type(grid, cell):
        """
        Gets information about the cell in (x,y). Walls and start pose are not
        traversable.
        
        :param cell: A tuple (x,y)
        :returns: A tuple (is_traversable, weight)
        """
        
        if "P" in grid[cell]:
            cell_info = (cell, True, 0, "P")
        else:
            cell_info = {
                "T": (cell, True, 0, "T"),
                "D": (cell, True, 2, "D"),
                "S": (cell, False, None, "S"),
                "G": (cell, True, 0, "G"),
                "": (cell, True, 0, ""),
                "o": (cell, True, 0, "o"),
                "W": (cell, False, None, "W")
            }.get(grid[cell])
        
        return cell_info
    
    @staticmethod
    def get_path_from(node):
        """Computes the upward path from a leaf node."""
        # print(node)
        path = [(node.pos, node.cell_type)]
        curr_node = node.parent
        while curr_node is not None:
            # print(curr_node)
            path.append((curr_node.pos, curr_node.cell_type))
            curr_node = curr_node.parent
            
        return path[::-1]  # same as .reverse()
    
    def a_star(self):
        """Runs the A* search for the grid setup"""
        node0 = PathNode(None, self.init, cell_type="S")
        nodeX = PathNode(None, self.end, cell_type="G")
        
        opened, closed = [node0], []
        
        while len(opened) > 0:
            # print("[0]: {} | [C]: {}".format(len(opened), len(closed)))
            # Get the node with smallest F cost
            costs = map(lambda p: p.f, opened)
            curr_node, curr_idx = [(n,i) for i, (n, f) in enumerate(zip(opened, costs)) if f == min(costs)][0]
            
            # Switch it to 'closed'
            closed.append(curr_node)
            opened.pop(curr_idx)
            
            if curr_node == nodeX:
                # The path has been found!
                # print(curr_node)
                
                path = PathFinder.get_path_from(curr_node)
                # print("A path has been found!\n{}".format(self.path))
                return path
                
            # Looks at the neighbor cells who are within grid range and are walkable (no walls).
            # Also cast each into a node
            block = PathFinder.walkable_node_neighbors(self.grid, curr_node, rad=1)
            
            # If the cell is not already closed and is walkable, proceed
            for node in block:
                if node not in closed:
                    node.g = curr_node.g + 1
                    # Euclidean distance used as metric (+ cell type weight)
                    node.h = sum([(a - b)**2 for (a,b) in zip(node.pos, curr_node.pos)]) + node.weight
                    node.f = node.g + node.h
                    
                    # If the node is in the "open" list and its new weight
                    # is more than the prev one, discard it
                    # here's the opposed case: there's no node like that
                    if [nopen for nopen in opened if (nopen == node and node.g > nopen.g)] == []:                            
                        opened.append(node)
        return []

In [78]:
print(g1)

[['' '' '' '' 'W']
 ['G' '' '' '' 'W']
 ['' '' '' '' 'W']
 ['T' '' '' 'T' 'D']
 ['P' 'o' 'o' 'P' 'S']]


In [46]:
print(subj_pos, goal_pos)

(0, 4) (3, 0)


In [83]:
path_finder = PathFinder(g1, subj_pos, goal_pos)
path = path_finder.a_star()
print(path)

[((4, 4), 'S'), ((4, 3), 'P'), ((3, 3), 'T'), ((2, 3), ''), ((1, 3), ''), ((1, 2), ''), ((1, 1), ''), ((1, 0), 'G')]


---
## Apply it to Sphinx!

Now we have two results and we want to apply them to Sphinx. What we've to do is:

  1. Create a `.path` file for the subject using the Pathfinder list of points.
  2. Create a `.world` file given the world's objects' and goal poses.
  3. Create one or multiple `.path` files for the pedestrians' paths.
  4. Upon executing Sphinx, set the drone pose. Upon taking off, the height can vary (above head, waist-height, knee-height)

In [21]:
from tempfile import mkdtemp

FDIR = mkdtemp(prefix="sphinx_", suffix="_worldconf")

In [48]:
print(g1)
print(path)

[['' 'T' 'P' 'D' 'S']
 ['' '' 'o' 'W' '']
 ['' 'T' 'o' 'W' '']
 ['G' '' 'o' 'W' 'T']
 ['' 'T' 'P' 'W' '']]
[(0, 4), (0, 3), (0, 2), (1, 2), (2, 2), (3, 2), (3, 1), (3, 0)]


In [49]:
ff = NamedTemporaryFile(mode="w", suffix=".path", dir=FDIR, delete=False)
ff.name

'/tmp/sphinx_op2vzd_worldconf/tmpDZzxBz.path'

In [23]:
PATH_TEMPLATE_START = """<?xml version="1.0" encoding="UTF-8"?>
<path name="{}">
<auto_start>{}</auto_start>
<delay_start>{}</delay_start>
<loop>{}</loop>
"""

PATH_TEMPLATE_END = "\n</path>"

In [135]:
from tempfile import NamedTemporaryFile

class PathGen:
    def __init__(self, fdir, points, track_models=False,
                 loop=False, delay_start=5.0, auto_start=True, is_ped=False):
        """
        Creates a pedestrian path. By default, the path is done by walking
        (speed is 1) without stop points.

        :param points: A list of tuples (position, cell_type, speed=1, stop_duration=0)
        :param track_models: If True, position is given by a model's name
        :param loop: If True, the path is played on a loop
        :param delay_start: An integer to set how late the path will be crossed
        :param auto_start: If True, the path is started right after the delay
        :param is_ped:
        """
        self.f = NamedTemporaryFile(mode="w", suffix=".path", dir=FDIR, delete=False)
        
        self.points = points
        self.auto_start = str(auto_start).lower()
        self.delay_start = str(delay_start)
        self.loop = str(loop).lower()
        self.track_models = track_models
        self.is_ped = is_ped
        
        self.write_to_template()

    def _point_tag(self):
        """Returns a point tag"""
        return self.point_tag

    def _waypoint_tag(self, vel, point, stop=0.0):
        """
        Stringify a point.

        :param vel:
        :param point:
        :param stop:
        """
        txt = "walk"
        if vel > 1.5:
            txt = "run"

        return """
        <waypoint>
            <animation>{}</animation>
            <velocity>{}</velocity>
            {}
            <stop_duration>{}</stop_duration>
        </waypoint>""".format(txt, vel, point, stop)

    def get_point_tag(self, point):
        if self.track_models:
            return "<model>{}</model>".format(point)
        else:
            return "<xy>{} {}</xy>".format(point[0], point[1])
        
    def get_move_dir(self, p1, p2):
        if abs(p1[0] - p2[0]) == 1:
            return "N"
        elif abs(p1[1] - p2[1]) == 1:
            return "W"
        
    def make_turn_by(self, dist, prev_point, vel, point, stop):
        move_dir = self.get_move_dir(prev_point, point)
        dist *= [-1,1][(randint(0,1))]
        if move_dir == "N":
            p = (point[0], point[1] + dist)
        else:
            p = (point[0] + dist, point[1])
        return self._waypoint_tag(vel, self.get_point_tag(p), stop)
    
    def write_to_template(self):
        """Writes the input data to the file template"""
        waypoints = []

        for (i, (point, ctype, vel, stop)) in enumerate(self.points):
            point_tag = self.get_point_tag(point)
            
            if ctype == "T":
                # first and last cells are not "T" (trees)
                waypoint_tag = self.make_turn_by(0.5, self.points[i-1][0], float(vel), point, float(stop))
                waypoints.append(waypoint_tag)
            else:
                waypoint_tag = self._waypoint_tag(float(vel), point_tag, float(stop))
                waypoints.append(waypoint_tag)

        if self.is_ped:
            (point, _, vel, stop) = self.points[0]
            point_tag = self.get_point_tag(point)
            waypoint_tag = self._waypoint_tag(float(vel), point_tag, float(stop))
            waypoints.append(waypoint_tag)
                
        txt_waypoints = "\n".join(waypoints)
        template = PATH_TEMPLATE_START.format(
            "tempPath", self.auto_start, self.delay_start, self.loop
        )
        
        temp_txt = template + txt_waypoints + PATH_TEMPLATE_END
        self.f.write(temp_txt)
        self.f.close()

    def get_path(self):
        """Returns the filepath of the path file."""
        return self.f.name


In [50]:
print(path)

[(0, 4), (0, 3), (0, 2), (1, 2), (2, 2), (3, 2), (3, 1), (3, 0)]


In [4]:
path = [(1, 0), (2, 1), (3, 1), (4, 2), (4, 3)]

In [111]:
points = [(point, ctype, 1.0, 0.0) for (point, ctype) in path]
subj_gen = PathGen(FDIR, points)
SUBJ_FPATH = subj_gen.get_path()

In [112]:
!code {SUBJ_FPATH}

In [129]:
peds_paths.items()

[('P0',
  {'loop': True, 'pos': ([(4, 0), (4, 1), (4, 2), (4, 3)], 4, [0, 1, 2, 3])})]

In [136]:
PED_FPATHS = []

for (name, data) in peds_paths.items():
    points = [(p, "P", 1.0, 0.0) for p in data["pos"][0]]
    generator = PathGen(FDIR, points, loop=data["loop"], delay_start=0.0, is_ped=True)
    PED_FPATHS.append((name, generator.get_path()))

In [137]:
PED_FPATHS

[('P0', '/tmp/sphinx__PWTZo_worldconf/tmpYUh4IM.path')]

---

In [52]:
WORLD_TEMPLATE_START = """
<?xml version="1.0"?>
<sdf version='1.5'>
  <world name='default'>

    <gui fullscreen='0'>
      <camera name='user_camera'>
        <pose frame=''>2.70464 -1.54753 3.05779 0 0.339643 2.2362</pose>
        <view_controller>orbit</view_controller>
        <projection_type>perspective</projection_type>
      </camera>
    </gui>

    <spherical_coordinates>
      <latitude_deg>46.414296</latitude_deg>
      <longitude_deg>6.928084</longitude_deg>
    </spherical_coordinates>

    <physics type="ode">
      <real_time_update_rate>1000</real_time_update_rate>
      <max_step_size>0.001</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <max_contacts>20</max_contacts>
      <gravity>0 0 -9.81</gravity>
      <magnetic_field>0.1062e-6 20.8038e-6 -43.2881e-6</magnetic_field>
      <ode>
        <solver>
          <type>world</type>
          <min_step_size>0.0001</min_step_size>
          <iters>50</iters>
          <precon_iters>0</precon_iters>
          <sor>1.4</sor>
          <use_dynamic_moi_rescaling>1</use_dynamic_moi_rescaling>
        </solver>
        <constraints>
          <cfm>0</cfm>
          <erp>0.2</erp>
          <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
          <contact_surface_layer>0.001</contact_surface_layer>
        </constraints>
      </ode>
    </physics>

    <atmosphere type="adiabatic">
      <temperature>298.15</temperature>
      <pressure>101325</pressure>
      <temperature_gradient>-0.0065</temperature_gradient>
    </atmosphere>

    <plugin name="fwman" filename="libsphinx_fwman.so">
      <spawn_point name="default">
        <pose>{}</pose>
      </spawn_point>
    </plugin>

    <!--*****************-->
    <!--Scene description-->
    <!--*****************-->
    <include>
      <uri>model://ground_plane</uri>
    </include>
"""                   

WORLD_TEMPLATE_END = """
    <scene>
      <ambient>0.4 0.4 0.4 1</ambient>
      <background>0.7 0.7 0.7 1</background>
      <shadows>1</shadows>
    </scene>    
    <light type="directional" name="sun">
      <cast_shadows>true</cast_shadows>
      <pose>0 0 400 0 0 0</pose>
      <diffuse>0.8 0.8 0.8 1</diffuse>
      <specular>0.2 0.2 0.2 1</specular>
      <attenuation>
        <range>1000</range>
        <constant>0.9</constant>
        <linear>0.01</linear>
        <quadratic>0.001</quadratic>
      </attenuation>
      <direction>-0.5 0.1 -0.9</direction>
    </light>
  </world>
</sdf>
"""

In [125]:
from tempfile import NamedTemporaryFile, mkdtemp

class WorldGen:
    def __init__(self, fdir, grid, goal_pos, subj_pos, drone_pos, peds_paths=None, tree_pos=None, door_pos=None, wall_pos=None):
        """
        Creates a .world file with the given parameters for objects and
        pedestrians. Pedestrians and objects are optional.

        :param grid: A 2d numpy array of dtype str with NxN shape
        :param goal_pos: A tuple (x,y)
        :param subj_pos: A tuple (x,y)
        :param drone_pos: A tuple (x,y)
        :param peds_paths: A dictionary with K-V {PedX: {pos: [(x0,y0),...], loop:True/False}}
        :param tree_pos: A list of (x,y) points
        :param door_pos: A list of (x,y, dir) points
        :param wall_pos: A list of (x,y) points
        """
        self.f = NamedTemporaryFile(mode="w", suffix=".world", dir=fdir, delete=False)
        
        self.goal_pos = WorldGen.world_pose(goal_pos[0], goal_pos[1])  # "paint it" in red
        self.subj_pos = WorldGen.world_pose(subj_pos[0], subj_pos[1])  # "paint it" in green
        self.drone_pos = WorldGen.world_pose(drone_pos[0], drone_pos[1], 0.1)

        if peds_paths is not None:
            self.peds_poses = []
            for opts in peds_paths.values():
                # opts has "pos" and "loop"
                # pos is (poses, row/col, list of x/y points)
                init_pos = opts["pos"][0][0]
                end_pos = opts["pos"][0][-1]
                self.peds_poses.append(WorldGen.world_pose(init_pos[0], init_pos[1]))
                self.peds_poses.append(WorldGen.world_pose(end_pos[0], end_pos[1]))
        
        # By default, 'None or []' is []
        self.tree_pos = tree_pos or []
        self.door_pos = door_pos or []
        self.wall_pos = wall_pos or []
            
        self.write_to_template()
        
    @staticmethod
    def world_pose(x, y, z=0.0):
        return "{} {} {} 0 0 0".format(x, y, z)
    
    def _object_tag(self, name, model, pose):
        """Returns an object 'include' tag"""

        return """
        <include>
          <name>{}</name>
          <uri>model://{}</uri>
          <pose>{}</pose>
        </include>
        """.format(name, model, pose)

    def _tree_tag(self, name, point):
        """"""
        canopy_pos = WorldGen.world_pose(point[0], point[1], 2.5)
        canopy_nm = "{}_canopy".format(name)
        wood_pos = WorldGen.world_pose(point[0], point[1])
        wood_nm = "{}_wood".format(name)
        
        tag = []
        tag.append(self._object_tag(canopy_nm, "tree_canopy", canopy_pos))
        tag.append(self._object_tag(wood_nm, "vertical_bar", wood_pos))
        return "\n".join(tag)
        
    def _door_tag(self, name, center):
        door_center = WorldGen.world_pose(center[0], center[1], 2.5)
        bar1 = WorldGen.world_pose(center[0] - 0.6, center[1])
        bar2 = WorldGen.world_pose(center[0] + 0.6, center[1])
        
        tag = []
        tag.append(self._object_tag("{}_rail".format(name), "top_door_rail", door_center))
        tag.append(self._object_tag("{}_bar1".format(name), "vertical_bar", bar1))
        tag.append(self._object_tag("{}_bar2".format(name), "vertical_bar", bar2))
        return "\n".join(tag)
    
    def _wall_tag(self, name, points):
        tag = []
        for (i, point) in enumerate(points):
            nm = "{}{}".format(name, i)
            box_pose = WorldGen.world_pose(point[0], point[1])
            tag.append(self._object_tag(nm, "wall_box", box_pose))
            
        return "\n".join(tag)
            
    def write_to_template(self):
        """"""
        includes = []

        includes.append(
            self._object_tag("start", "ground_red", self.subj_pos)
        )
        includes.append(
            self._object_tag("goal", "ground_goal", self.goal_pos)
        )
        
        for (i, point) in enumerate(self.peds_poses):
            includes.append(self._object_tag("PX{}".format(i), "ground_tour", point))
        for (i, point) in enumerate(self.tree_pos):
            includes.append(self._tree_tag("T{}".format(i), point))
            
        for (i, point) in enumerate(self.door_pos):
            includes.append(self._door_tag("D{}".format(i), point))
            
        for (i, points) in enumerate(self.wall_pos):
            # for each wall, being a wall a list of points
            includes.append(self._wall_tag("W{}".format(i), points))
        
        objs_includes = "\n".join(includes)

        template = WORLD_TEMPLATE_START.format(self.drone_pos)
        temp_txt = template + objs_includes + WORLD_TEMPLATE_END
        self.f.write(temp_txt)
        self.f.close()
        
    def get_path(self):
        return self.f.name


In [ ]:
# :param grid: A 2d numpy array of dtype str with NxN shape
# :param goal_pos: A tuple (x,y)
# :param subj_pos: A tuple (x,y)
# :param drone_pos: A tuple (x,y)
# :param peds_paths: A dictionary with K-V {PedX: {pos: [(x0,y0),...], loop:True/False}}
# :param tree_pos: A list of (x,y) points
# :param door_pos: A list of (x,y, dir) points
# :param wall_pos: A list of (x,y) points

In [72]:
trees

{'T0': {'pos': array([3, 3])}, 'T1': {'pos': array([3, 0])}}

In [105]:
tree_pts = [tuple(val["pos"]) for val in trees.values()]
wall_pts = [walls.values()[0]["pos"][0]]  # a list of walls

In [55]:
print(goal_pos)
print(subj_pos)
print(drone_pos)

print(tree_pts)
print(doors)  # None

print(wall_pts)
print(peds_paths)

(3, 0)
(0, 4)
(0, 3)
[(0, 1), (2, 1)]
{}
[[(1, 3), (2, 3), (3, 3), (4, 3)]]
{'P0': {'pos': ([(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)], [0, 1, 2, 3, 4], 2), 'loop': True}}


In [74]:
g1

array([['', '', '', '', 'W'],
       ['G', '', '', '', 'W'],
       ['', '', '', '', 'W'],
       ['T', '', '', 'T', 'D'],
       ['P', 'o', 'o', 'P', 'S']], dtype='|S1')

In [126]:
world_gen = WorldGen(FDIR, g1, goal_pos, subj_pos, drone_pos, peds_paths, tree_pts, None, wall_pts)
WORLD_FPATH = world_gen.get_path()

In [127]:
!code {WORLD_FPATH}

In [31]:
os.system("sphinx {} $SPHINX/drones/local_bebop2.drone".format(WORLD_FPATH))

0

In [38]:
!sphinx /tmp/tmpJJeNEb.world /opt/parrot-sphinx/usr/share/sphinx/drones/local_bebop2.drone

Parrot-Sphinx simulator version 1.4

connecting to firmwared version: 1.4
starting Telemetryd to capture true data...
I telemetryd: Creating daemon with rate 50 ms
I libtelemetry: clear logger filter
I telemetryd: Loading plugins from '/opt/parrot-sphinx/usr/bin/../lib/tlm-plugins/'
I telemetryd: Loading '/opt/parrot-sphinx/usr/bin/../lib/tlm-plugins//tlm-blackbox.so'
I tlmblackbox: setEnabled 1
I tlmblackbox: setCompressed 1
I tlmblackbox: setCompressionLevel 6
I tlmblackbox: setCompressionMethod zlib
I tlmblackbox: setLogDir '.'
I tlmblackbox: setLogName 'log.tlmb'
I tlmblackbox: setLogCount 4
I tlmblackbox: setFilter *
I libtelemetry: clear logger filter
I tlmblackbox: setFlushPeriod 10000 ms
I tlmblackbox: setMaxSize -1 bytes
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.3' -> './sphinx-11345-log.tlmb.4'
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.2' -> './sphinx-11345-log.tlmb.3'
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.1' -> './sphinx-11345-log.tlmb.2'

In [52]:
# !rm -R {FDIR}

In [73]:
from subprocess import Popen

cmd = "sphinx {} /opt/parrot-sphinx/usr/share/sphinx/drones/local_bebop2.drone".format(WORLD_FPATH).split(" ")
stdout = Popen(cmd)

In [150]:
SPHINX_ROOT = os.getenv("SPHINX_ROOT")
DRONE_FPATH = "{}drones/local_bebop2.drone".format(SPHINX_ROOT)
SUBJ_PATH = "{}actor/pedestrian.actor::name=subject::path={} ".format(SPHINX_ROOT, SUBJ_FPATH)
PED1_PATH = "{}actor/pedestrian.actor::name=pedestrian{}::path={} "

'/opt/parrot-sphinx/usr/share/sphinx/'

In [ ]:
for p in range(no_peds):
    PED_PATHS += PED1_PATH.format(SPHINX_ROOT, p, peds[p]["fpath"])
!sphinx {WORLD_PATH} {DRONE_PATH} {PED_PATHS} &

In [138]:
from subprocess import Popen

SPHINX = os.getenv("SPHINX_ROOT")
_drone = "/opt/parrot-sphinx/usr/share/sphinx/drones/local_bebop2.drone"
_subject = "/opt/parrot-sphinx/usr/share/sphinx/actors/pedestrian.actor::name=subject::path={}".format(SUBJ_FPATH)
_peds = " ".join(["/opt/parrot-sphinx/usr/share/sphinx/actors/pedestrian.actor::name={}::path={}".format(nm, pth) for (nm, pth) in PED_FPATHS])

cmd = "sphinx {} {} {} {}".format(WORLD_FPATH, _drone, _subject, _peds).split(" ")
stdout = Popen(cmd)

In [99]:
print(WORLD_FPATH)
print(SUBJ_FPATH)

/tmp/sphinx__PWTZo_worldconf/tmpzMPJLi.world
/tmp/sphinx__PWTZo_worldconf/tmprgdJKG.path


In [37]:
SPHINX

'/opt/parrot-sphinx/usr/share/sphinx/'

In [113]:
from subprocess import Popen

SPHINX = os.getenv("SPHINX_ROOT")
_drone = "/opt/parrot-sphinx/usr/share/sphinx/drones/local_bebop2.drone"
_actor = "/opt/parrot-sphinx/usr/share/sphinx/actors/pedestrian.actor::name=subject::path={}".format(SUBJ_FPATH)

cmd = "sphinx {} {} {}".format(WORLD_FPATH, _drone, _actor).split(" ")
stdout = Popen(cmd)

In [40]:
print(WORLD_FPATH)
print(_drone)
print(_actor)

/tmp/sphinx__PWTZo_worldconf/tmpmWgaU7.world
/opt/parrot-sphinx/usr/share/sphinx/drones/local_bebop2.drone
/opt/parrot-sphinx/usr/share/sphinx/actor/pedestrian.actor::name=subject::path=/tmp/sphinx__PWTZo_worldconf/tmpngsCj3.path


In [39]:
!sphinx {WORLD_FPATH} {_drone} {_actor}

Parrot-Sphinx simulator version 1.4

connecting to firmwared version: 1.4
starting Telemetryd to capture true data...
I telemetryd: Creating daemon with rate 50 ms
I libtelemetry: clear logger filter
I telemetryd: Loading plugins from '/opt/parrot-sphinx/usr/bin/../lib/tlm-plugins/'
I telemetryd: Loading '/opt/parrot-sphinx/usr/bin/../lib/tlm-plugins//tlm-blackbox.so'
I tlmblackbox: setEnabled 1
I tlmblackbox: setCompressed 1
I tlmblackbox: setCompressionLevel 6
I tlmblackbox: setCompressionMethod zlib
I tlmblackbox: setLogDir '.'
I tlmblackbox: setLogName 'log.tlmb'
I tlmblackbox: setLogCount 4
I tlmblackbox: setFilter *
I libtelemetry: clear logger filter
I tlmblackbox: setFlushPeriod 10000 ms
I tlmblackbox: setMaxSize -1 bytes
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.3' -> './sphinx-11345-log.tlmb.4'
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.2' -> './sphinx-11345-log.tlmb.3'
I tlmblackbox: rotate logs: './sphinx-11345-log.tlmb.1' -> './sphinx-11345-log.tlmb.2'

In [11]:
from tempfile import NamedTemporaryFile, mkdtemp
# from templates import PATH_TEMPLATE_START, PATH_TEMPLATE_END, WORLD_TEMPLATE_START, WORLD_TEMPLATE_END

class RandomSphinxWorld:
    
    def __init__(self, world_conf, subject_path, peds_paths):
        """
        Creates Sphinx-related world configuration files from the generated world setup.
        
        :param world_conf: A dict with keys (...)
        :param subject_path: A list of (x,y) coordinates
        :param peds_paths: A set of lists of (x,y) coordinates
        """
        fdir = mkdtemp(prefix="sphinx_gen")
        
        self.subject_path = RandomSphinxWorld.create_path(fdir, subject_path)
        self.ped_paths = dict()
        
        for (pname, ppath) in peds.items():
            # ppath is a dict with keys ("path", "loop"). No delay_start nor stop_duration nor speed.
            self.ped_paths[pname] = RandomSphinxWorld.create_path(fdir, ppath)
            
        self.world_path = RandomSphinxWorld.create_world(fdir, world_conf)
        
    @staticmethod
    def create_path(fdir, path):
        ff = NamedTemporaryFile(dir=fdir, suffix=".path")
        
        while False:
            ff.write(".")

    @staticmethod
    def create_world(fdir, conf):
        ff = NamedTemporaryFile(dir=fdir, suffix=".path")
        
        while False:
            ff.write(".")


In [ ]:
obj_probs = {"tree": 0.4, "door": 0.0, "wall": 0.2}

my_world = RandomWorld(grid=(5,5), goal_dist=4, drone_dist=1, no_peds=1, probs=obj_probs)
my_path = PathFinder(my_world)

# RandomSphinx.remove_last()  # removes all the generated files in the previous run

generator = RandomSphinxWorld(my_world, my_path, my_world.ped_paths)
# world_fpath = generator.world_fpath
# subj_fpath = generator.subj_fpath
# peds_fpaths = generator.peds_fpaths
# "/tmp/worldpath.txt" is set to generator.world_fpath

heights = {
    "above": 1.8,
    "mid": 1.0,
    "below": 0.6
}

generator.init(drone_height="above")  # wait() until it finishes

generator.close_log()  # closes the .csv log